In [1]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
# Read file
city_proper = gpd.read_file('./data/city_proper_over_1000.geojson')
hsr_countries = [
    "China",
    "Spain",
    "France",
    "Germany",
    "Japan",
    "Italy",
    "United Kingdom",
    "Korea, Republic of",
    "Turkey",
    "Finland",
    "Sweden",
    "Uzbekistan",
    "United States",
    "Greece",
    "Russia",
    "Saudi Arabia",
    "Taiwan, China",
    "Austria",
    "Portugal",
    "Poland",
    "Belgium",
    "Morocco",
    "Switzerland",
    "Indonesia",
    "Norway",
    "Netherlands",
    "Serbia",
    "Denmark",
    "Hong Kong, China",
]

In [3]:
city_proper = city_proper[
    [
        "geoname_id",
        "name",
        "ascii_name",
        "country_code",
        "cou_name_en",
        "population",
        "geometry",
    ]
]

city_proper["geoname_id"] = city_proper["geoname_id"].astype("string")
city_proper["name"] = city_proper["name"].astype("string")
city_proper["ascii_name"] = city_proper["ascii_name"].astype("string")
city_proper["country_code"] = city_proper["country_code"].astype("string")
city_proper["cou_name_en"] = city_proper["cou_name_en"].astype("string")
city_proper["population"] = city_proper["population"].astype(int)

In [4]:
city_proper["cou_name_en"] = city_proper["cou_name_en"].fillna("")
city_proper = city_proper[city_proper["cou_name_en"].apply(lambda x: x in hsr_countries)].reset_index(drop=True)
city_proper.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 79439 entries, 0 to 79438
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   geoname_id    79439 non-null  string  
 1   name          79439 non-null  string  
 2   ascii_name    79439 non-null  string  
 3   country_code  79439 non-null  string  
 4   cou_name_en   79439 non-null  string  
 5   population    79439 non-null  int32   
 6   geometry      79439 non-null  geometry
dtypes: geometry(1), int32(1), string(5)
memory usage: 3.9 MB


In [5]:
# Take a subset of only cities with population greater than 50K
city_over_50 = city_proper[city_proper["population"] >= 32000]
len(city_over_50)

6856

In [6]:
# Want to calculate dist between every city over 50K and every city below 50K
crossed = city_over_50.merge(
    city_over_50,
    how="cross",
)
crossed = gpd.GeoDataFrame(crossed)

In [7]:
crossed.set_geometry("geometry_x")
crossed["geometry_x"] = gpd.GeoSeries(crossed["geometry_x"]).to_crs("epsg:3857")
crossed["geometry_y"] = gpd.GeoSeries(crossed["geometry_y"]).to_crs("epsg:3857")
crossed["dist"] = crossed["geometry_x"].distance(crossed["geometry_y"])

In [8]:
crossed["dist"] = crossed["dist"].astype(int)
crossed = crossed[crossed["name_x"] != crossed["name_y"]]

In [9]:
# Save smaller dataset for later if needed
# crossed[crossed["dist"] < 80467.2].to_csv("./data/under_50_mi.csv")

In [10]:
# READ CHECKPOINT
# crossed = gpd.read_file('./data/under_50_mi.csv')
# crossed = crossed[crossed["ascii_name_x"] != crossed["ascii_name_y"]]
# crossed["population_x"] = crossed["population_x"].astype(int)
# crossed["population_y"] = crossed["population_y"].astype(int)
# crossed["dist"] = crossed["dist"].astype(float)

In [11]:
# Select for city pairs closer than 50 miles to main city (over 50K)
valid = crossed[crossed["dist"] < 80000]
# valid = crossed

# If the second (smaller) city is matched to multiple main cities, only keep match with largest main city

valid = valid.sort_values(by="population_x", ascending=False)
valid = valid.drop_duplicates(subset="name_y", keep="first")

valid.head(2)

,geoname_id_x,name_x,ascii_name_x,country_code_x,cou_name_en_x,population_x,geometry_x,geoname_id_y,name_y,ascii_name_y,country_code_y,cou_name_en_y,population_y,geometry_y,dist
23279119,1796236,Shanghai,Shanghai,CN,China,22315474,POINT (13520649.392 3661642.393),1794035,Songjiang,Songjiang,CN,China,130218,POINT (13494511.576 3637221.342),35771
23276529,1796236,Shanghai,Shanghai,CN,China,22315474,POINT (13520649.392 3661642.393),1806840,Huilong,Huilong,CN,China,74818,POINT (13542572.652 3738541.339),79962


In [12]:
# Groupby main city, sum population, agg list of names together for record and keep largest city as metro name
valid = (
    valid.groupby("ascii_name_x")
    .agg(
        {
            "population_x": "first",
            "country_code_x": "first",
            "geometry_x": "first",
            "population_y": "sum",
            "ascii_name_y": list,
        }
    )
    .reset_index()
)
valid["population"] = valid["population_x"] + valid["population_y"]
valid = valid.rename(columns={
    "ascii_name_x": "central city",
    "population_x": "central city pop",
    "country_code_x": "country_code",
    "population": "total pop",
    "population_y": "population",
    "geometry_x": "geometry"
})
valid["population"] = valid["population"].astype(int)

In [13]:
valid = valid.sort_values(by="total pop", ascending=False)

In [14]:
# Efficiently determine rows to drop
seen_prev = set()  # To keep track of all values seen in column 'B'
rows_to_keep = []  # List to keep track of row indices that we do not want to drop

for index, row in valid.iterrows():
    # Check if the current value in 'A' has been seen in any 'B' list before this row
    if row["central city"] not in seen_prev:
        rows_to_keep.append(index)
    # Add the current row's 'B' list items to the seen set
    seen_prev.update(row["ascii_name_y"])

# Filter the DataFrame to keep only the rows we want
valid_dropped = valid.loc[rows_to_keep].reset_index(drop=True)

In [15]:
valid_dropped.head(20)

,central city,central city pop,country_code,geometry,population,ascii_name_y,total pop
0,Shenzhen,17494398,CN,POINT (12698025.072 2577151.127),36825407,"[Nam Cheong, Wan Chai, Tseung Kwan O, Tai Koo,...",54319805
1,Tokyo,8336599,JP,POINT (15550410.025 4257980.732),23736774,"[Oami, Higashikurume, Wako, Ushiku, Togane, Zu...",32073373
2,Langfang,868066,CN,POINT (12992622.085 4796547.206),30051058,"[Tianjin, Beijing]",30919124
3,Shanghai,22315474,CN,POINT (13520649.392 3661642.393),4165757,"[Songjiang, Huilong, Taicang, Zhabei, Jiashan,...",26481231
4,New York City,8804190,US,POINT (-8238306.896 4970287.468),16427587,"[The Bronx, Woodhaven, Hoboken, Wakefield, Pis...",25231777
5,Jakarta,8540121,ID,POINT (11893945.465 -693168.833),13913082,"[Depok, Cikupa, Caringin, Tangerang, Sukabumi,...",22453203
6,Seoul,10349312,KR,POINT (14135170.830 4518296.290),11834000,"[Icheon-si, Seongnam-si, Guri-si, Pubal, Uijeo...",22183312
7,Istanbul,14804116,TR,POINT (3222661.410 5014383.275),6801193,"[UEskuedar, Maltepe, Eminoenue, Bahcelievler, ...",21605309
8,Wuxi,4396835,CN,POINT (13390462.361 3706850.215),13071777,"[Jiangyin, Yixing, Changzhou, Suzhou]",17468612
9,London,8961989,GB,POINT (-13997.313 6711744.580),8214498,"[Islington, Gillingham, Canary Wharf, Canvey I...",17176487


In [16]:
world = gpd.read_file("./data/countries.geojson")
world = world.to_crs("epsg:4326")

In [17]:
df = valid_dropped
df = df.set_geometry("geometry")
df = df.set_crs("epsg:3857", allow_override=True)
df = df.to_crs("epsg:4326")
px.scatter_geo(
    df,
    lat=df.geometry.y,
    lon=df.geometry.x,
    hover_data=["central city", "total pop"],
)

In [23]:
valid_dropped.sample(2)

,central city,central city pop,country_code,geometry,population,ascii_name_y,total pop
528,Fredericia,40886,DK,POINT (1085651.126 7472436.238),180863,[Odense],221749
243,Bilbao,345821,ES,POINT (-325640.680 5352044.860),708770,"[Algorta, Basauri, Santutxu, Gasteiz / Vitoria...",1054591


In [24]:
valid_dropped = valid_dropped.drop(columns=["central city pop", "total pop"])
valid_dropped = valid_dropped.rename(
    columns={"central city": "metro", "ascii_name_y": "cities"}
)
valid_dropped.sample(2)

,metro,country_code,geometry,population,cities
156,Virginia Beach,US,POINT (-8457831.158 4418626.904),1312755,"[South Suffolk, Newport News, Suffolk, Hampton..."
533,Menglang,CN,POINT (11121962.608 2579152.063),128774,"[Zhutang, Qianliu, Shangyun]"


In [27]:
valid_dropped["geometry"] = valid_dropped["geometry"].astype("string")
valid_dropped.to_parquet("./data/metro_regions.parquet")

In [18]:
q

NameError: name 'q' is not defined

In [ ]:
valid_cross = gpd.GeoDataFrame(valid.merge(
    valid,
    how="cross"
))
valid_cross.sort_values(by="total pop_x", ascending=False).head(2)

In [ ]:
valid_cross.set_geometry("geometry_x")
valid_cross["geometry_x"] = gpd.GeoSeries(valid_cross["geometry_x"]).to_crs("epsg:3857")
valid_cross["geometry_y"] = gpd.GeoSeries(valid_cross["geometry_y"]).to_crs("epsg:3857")
valid_cross["dist"] = valid_cross["geometry_x"].distance(valid_cross["geometry_y"])

In [ ]:
valid_cross = valid_cross[
    valid_cross["central city_x"] != valid_cross["central city_y"]
]
valid_cross["total pop_x"] = valid_cross["total pop_x"].astype(int)
valid_cross["total pop_y"] = valid_cross["total pop_y"].astype(int)
valid_cross["dist"] = valid_cross["dist"].astype(float)

In [ ]:
# Select for city pairs closer than 20 miles to main city (over 75K)
valid_cross = valid_cross[valid_cross["dist"].astype("float") < 50000]
# valid = crossed
# If the second (smaller) city is matched to multiple main cities, only keep match with largest main city
valid_cross = valid_cross.sort_values(by="central city pop_x", ascending=False)
valid_cross = valid_cross.drop_duplicates(subset="central city_y", keep="first")
valid_cross.head(2)

In [ ]:
# Groupby main city, sum population, agg list of names together for record and keep largest city as metro name
valid_cross = (
    valid_cross.groupby("central city_x")
    .agg(
        {
            "total pop_x": "first",
            "country_code_x": "first",
            "geometry_x": "first",
            "total pop_y": "sum",
            "central city_y": list,
        }
    )
    .reset_index()
)
valid_cross["total pop"] = valid_cross["total pop_x"] + valid_cross["total pop_y"]
valid_cross = valid_cross.rename(
    columns={
        "central_city_x": "central city",
        "total pop_x": "central city pop",
        "country_code_x": "country_code",
        "total pop_y": "total pop other",
        "population_y": "population",
        "geometry_x": "geometry",
    }
)
valid_cross["total pop"] = valid_cross["total pop"].astype(int)

In [ ]:
valid_cross.sample(2)

In [ ]:
valid_cross.sort_values(by="total pop", ascending=False).head(25)

In [ ]:
valid_cross[
    ~valid_cross.apply(lambda row: row["central city_x"] in row["central city_y"], axis=1)
].sort_values(by="total pop", ascending=False).head(25)

In [ ]:
world = gpd.read_file("./data/countries.geojson")
world = world.to_crs("epsg:4326")

In [ ]:
world = gpd.read_file("./data/countries.geojson")
world = world.to_crs("epsg:4326")

valid_cross = valid_cross.set_geometry("geometry")
valid_cross = valid_cross = valid_cross.set_crs("epsg:4326")
px.scatter_geo(
    valid_cross,
    lat=valid_cross.geometry.y,
    lon=valid_cross.geometry.x,
    hover_data=["central city_x"],
)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
world.plot(ax=ax, color="lightgrey", markersize=2)
valid_geo.plot(ax=ax, color="red", markersize=0.25, labels="name_x")
ax.set_aspect("equal")
plt.show()

In [ ]:
valid[valid["name_x"].str.contains("Atlanta")]